## APLICACION PARA CREAR LA BASE DE DE DATOS

In [ ]:
# =============================================================================
# PROGRAMA 1: CREADOR DE LA BASE DE DATOS
# =============================================================================

from sqlalchemy import create_engine, text
import urllib

# --- Parámetros de conexión ---
SERVER_NAME = r'IBAI\SQLEXPRESS'
DB_TO_CREATE = 'RETO0'
SYSTEM_DB = 'master'  # Conectarse a 'master' para crear otras bases de datos

# --- Conexión al servidor ---
engine = None
try:
    connection_string = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={SERVER_NAME};"
        f"DATABASE={SYSTEM_DB};"
        f"Trusted_Connection=yes;"
    )
    quoted_conn_str = urllib.parse.quote_plus(connection_string)
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted_conn_str}")

    print(f"✅ Conexión exitosa con el servidor '{SERVER_NAME}' (usando la BBDD 'master').")

except Exception as e:
    print(f"❌ ERROR: No se pudo conectar al servidor. Verifica el nombre del servidor y que esté en ejecución.")
    print(f"   Detalle del error: {e}")

# --- Ejecución del comando para crear la base de datos ---
if engine is not None:
    try:
        sql_create_db_command = f"""
        IF NOT EXISTS (SELECT name FROM sys.databases WHERE name = N'{DB_TO_CREATE}')
        BEGIN
            CREATE DATABASE [{DB_TO_CREATE}];
        END;
        """
        with engine.connect() as connection:
            connection.execution_options(isolation_level="AUTOCOMMIT").execute(text(sql_create_db_command))

        print(f"✔️ ¡Éxito! La base de datos '{DB_TO_CREATE}' ha sido creada (o ya existía).")

    except Exception as e:
        print(f"❌ ERROR durante la creación de la base de datos: {e}")

## APLICACION PARA BORRAR LAS TABLAS

In [ ]:
# =============================================================================
# PROGRAMA 2: BORRADOR DE TABLAS
# =============================================================================

from sqlalchemy import create_engine, text
import urllib

# --- Parámetros de conexión ---
SERVER_NAME = r'IBAI\SQLEXPRESS'
DATABASE_NAME = 'RETO0'  # Base de datos actualizada

# --- Conexión a la base de datos RETO0 ---
try:
    connection_string = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={SERVER_NAME};"
        f"DATABASE={DATABASE_NAME};"
        f"Trusted_Connection=yes;"
    )
    quoted_conn_str = urllib.parse.quote_plus(connection_string)
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted_conn_str}")

    print(f"✅ Conexión exitosa con la BBDD '{DATABASE_NAME}' para borrar tablas.")

    # --- Ejecutar comandos SQL para borrar las tablas ---
    with engine.connect() as connection:
        print("\n🔄 Intentando borrar las tablas...")

        connection.execute(text("DROP TABLE IF EXISTS stocks"))
        print("   -> Tabla 'stocks' borrada con éxito (o no existía).")

        connection.execute(text("DROP TABLE IF EXISTS cryptomonedas"))
        print("   -> Tabla 'cryptomonedas' borrada con éxito (o no existía).")

        connection.commit()

    print("\n🏁 Proceso de borrado finalizado.")

except Exception as e:
    print(f"❌ ERROR durante el proceso de borrado: {e}")

## APLICACION PARA CREAR LAS TABLAS

In [ ]:
# =============================================================================
# PROGRAMA 3: CREADOR DE ESTRUCTURA DE TABLAS (VACÍAS)
# =============================================================================
import pandas as pd
from sqlalchemy import create_engine
import urllib

SERVER_NAME = r'IBAI\SQLEXPRESS'
DATABASE_NAME = 'RETO0'

try:
    connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER_NAME};DATABASE={DATABASE_NAME};Trusted_Connection=yes;"
    quoted_conn_str = urllib.parse.quote_plus(connection_string)
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted_conn_str}")
    print(f"✅ Conexión exitosa con la BBDD '{DATABASE_NAME}' para crear estructuras.")

    def crear_tabla_vacia_desde_csv(ruta_csv, nombre_tabla):
        print(f"\n🔄 Creando estructura para la tabla '{nombre_tabla}'...")
        # Leemos el CSV para que pandas infiera el esquema (columnas y tipos)
        df_esquema = pd.read_csv(ruta_csv).head(0)
        # Creamos la tabla usando el esquema vacío. 'replace' asegura que se cree limpia.
        df_esquema.to_sql(nombre_tabla, engine, if_exists='replace', index=False)
        print(f"✔️ ¡Éxito! La tabla '{nombre_tabla}' ha sido creada vacía.")

    print("\n🚀 INICIANDO EL SCRIPT DE CREACIÓN DE ESTRUCTURAS...")
    crear_tabla_vacia_desde_csv('stocks.csv', 'stocks')
    crear_tabla_vacia_desde_csv('cryptocurrency.csv', 'cryptomonedas')
    print("\n🏁 Proceso de creación de estructuras finalizado.")

except Exception as e:
    print(f"❌ ERROR: {e}")

## APLICACION PARA INSERTAR DATOS

In [1]:
# =============================================================================
# PROGRAMA 4: INSERCIÓN DE DATOS EN TABLAS EXISTENTES (VERSIÓN REVISADA)
# =============================================================================
import pandas as pd
from sqlalchemy import create_engine, text
import urllib

SERVER_NAME = r'IBAI\SQLEXPRESS'
DATABASE_NAME = 'RETO0'

try:
    connection_string = f"DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={SERVER_NAME};DATABASE={DATABASE_NAME};Trusted_Connection=yes;"
    quoted_conn_str = urllib.parse.quote_plus(connection_string)
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted_conn_str}")
    print(f"✅ Conexión exitosa con la BBDD '{DATABASE_NAME}' para insertar datos.")

    def insertar_datos_en_tabla_existente(ruta_csv, nombre_tabla):
        print(f"\n🔄 Insertando datos en la tabla '{nombre_tabla}'...")

        # --- PASO 1: VACIAR LA TABLA (MÉTODO MÁS ROBUSTO) ---
        # Nos conectamos y ejecutamos TRUNCATE con autocommit para asegurar que se complete inmediatamente.
        with engine.connect() as connection:
            print(f"   -> Vaciando tabla '{nombre_tabla}'...")
            connection.execution_options(isolation_level="AUTOCOMMIT").execute(text(f"TRUNCATE TABLE {nombre_tabla}"))

        # --- PASO 2: INSERTAR LOS NUEVOS DATOS ---
        # Leemos el CSV y lo añadimos a la tabla ya existente y vacía.
        # if_exists='append' es correcto aquí porque la tabla ya está vacía.
        df_datos = pd.read_csv(ruta_csv)
        df_datos.to_sql(nombre_tabla, engine, if_exists='append', index=False)
        print(f"✔️ ¡Éxito! {len(df_datos):,} filas insertadas en '{nombre_tabla}'.".replace(',', '.'))

    print("\n🚀 INICIANDO EL SCRIPT DE INSERCIÓN DE DATOS...")
    insertar_datos_en_tabla_existente('stocks.csv', 'stocks')
    insertar_datos_en_tabla_existente('cryptocurrency.csv', 'cryptomonedas')
    print("\n🏁 Proceso de inserción finalizado.")

except Exception as e:
    print(f"❌ ERROR: {e}")

✅ Conexión exitosa con la BBDD 'RETO0' para insertar datos.

🚀 INICIANDO EL SCRIPT DE INSERCIÓN DE DATOS...

🔄 Insertando datos en la tabla 'stocks'...
   -> Vaciando tabla 'stocks'...
✔️ ¡Éxito! 156.878 filas insertadas en 'stocks'.

🔄 Insertando datos en la tabla 'cryptomonedas'...
   -> Vaciando tabla 'cryptomonedas'...
✔️ ¡Éxito! 97.846 filas insertadas en 'cryptomonedas'.

🏁 Proceso de inserción finalizado.


## APLICACION PARA CREAR LAS TABLAS E INSERTAR DATOS A LA VEZ

In [ ]:
# =============================================================================
# PROGRAMA 3: CREADOR DE TABLAS E INSERCIÓN DE DATOS DESDE CSV
# =============================================================================

import pandas as pd
from sqlalchemy import create_engine
import urllib

# --- Parámetros de conexión ---
SERVER_NAME = r'IBAI\SQLEXPRESS'
DATABASE_NAME = 'RETO0'  # Base de datos actualizada

# --- Conexión a la base de datos RETO0 ---
try:
    connection_string = (
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={SERVER_NAME};"
        f"DATABASE={DATABASE_NAME};"
        f"Trusted_Connection=yes;"
    )
    quoted_conn_str = urllib.parse.quote_plus(connection_string)
    engine = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted_conn_str}")
    print(f"✅ Conexión exitosa con la BBDD '{DATABASE_NAME}' para crear/cargar tablas.")
except Exception as e:
    print(f"❌ ERROR: No se pudo establecer la conexión: {e}")
    engine = None

# --- Función de carga de datos ---
def cargar_csv_a_sql(ruta_archivo_csv, nombre_tabla_destino):
    if engine is None:
        return
    try:
        print(f"\n🔄 Procesando '{ruta_archivo_csv}'...")
        df = pd.read_csv(ruta_archivo_csv)
        print(f"   -> Lectura completada: {len(df):,} filas.".replace(',', '.'))
        print(f"   -> Creando/cargando la tabla '{nombre_tabla_destino}'...")
        df.to_sql(nombre_tabla_destino, engine, if_exists='replace', index=False)
        print(f"✔️ ¡Éxito! La tabla '{nombre_tabla_destino}' ha sido creada y los datos cargados.")
    except Exception as e:
        print(f"❌ ERROR durante la carga de '{ruta_archivo_csv}': {e}")

# --- Ejecución del proceso de creación e inserción ---
if __name__ == "__main__" and engine is not None:
    print("\n🚀 INICIANDO EL SCRIPT DE CREACIÓN/CARGA DE TABLAS...")
    cargar_csv_a_sql('stocks.csv', 'stocks')
    cargar_csv_a_sql('cryptocurrency.csv', 'cryptomonedas')
    print("\n🏁 Proceso de carga finalizado. Puedes verificar las tablas en SQL Server.")